# Building the model
## Dependencies

In [3]:
#!pip install tensorflow==2.10 opencv-python matplotlib

In [4]:
# Functional API
import os
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [5]:
# We don't see any GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) == 0

## Data Loading & preprocessing

In [6]:
SAMPLE_SIZE = 300
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [7]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(SAMPLE_SIZE)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(SAMPLE_SIZE)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(SAMPLE_SIZE)

In [8]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img / 255.0
    return img

In [9]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negative = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negative)

In [10]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [11]:
# DataLoader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [12]:
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [13]:
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*.3))

## Model building

### Embedding

In [38]:
def make_embedding():
    inp = Input(shape=(100, 100, 3), name='input_image')
    
    conv1 = Conv2D(64, (10, 10), activation='relu')(inp)
    max_pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv1)
    
    conv2 = Conv2D(128, (7, 7), activation='relu')(max_pool1)
    max_pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv2)
    
    conv3 = Conv2D(128, (4, 4), activation='relu')(max_pool2)
    max_pool3 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv3)
    
    conv4 = Conv2D(256, (4, 4), activation='relu')(max_pool3)
    feature_vector = Flatten()(conv4)
    fc_1 = Dense(4096, activation='sigmoid')(feature_vector)
    
    return Model(inputs=[inp], outputs=[fc_1], name='embedding')

### Distance layer

In [43]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

### Siamese model

In [48]:
embedding = make_embedding()
l1 = L1Dist()

In [59]:
def make_siamese_model():
    input_image = Input(name='input_img', shape=(100, 100, 3))
    validation_image = Input(name='validation_img', shape=(100, 100, 3))
    
    input_embedding = embedding(input_image)
    validation_embedding = embedding(validation_image)
    
    distances = l1(input_embedding, validation_embedding)
    
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=[classifier], name='SiameseNetwork')

In [57]:
final_model = make_siamese_model()

In [58]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][0]']   